In [22]:
import os
import json
import tiktoken
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field

In [4]:
class Scenic_output(BaseModel):
    '''
    The output of Scenic language can be divided into the following
    fields:
    - Set Map and Model
    - Constants
        - Constants used in the following fields
    - Monitors
        - Traffic Light Monitors
    - Defining Agent Behaviors
        - Ego Behaviors
        - Other Vehicle Behaviors (Leading Cars, Following Cars, 
        Adversary Cars, Adjacent Cars, Pedestrians, etc.)
    - Defining Spatial Relations
        - Geometry
        - Placement
    - Scenario Specifications
        - Scenario Specifications
    - Background Activities
        - Defining Background Cars and Pedestrians
    - 
    '''
    map_and_model: str = Field(
        ...,
        title="Map and Model",
        description="The map and model of the scenario"
    )
    constants: str = Field(
        ...,
        title="Constants",
        description="Constants used in the scenario"
    )
    monitors: str = Field(
        ...,
        title="Monitors",
        description="Traffic Light Monitors"
    )
    behaviors: str = Field(
        ...,
        title="Behaviors",
        description="Defining Agent Behaviors"
    )
    spatial_relations: str = Field(
        ...,
        title="Spatial Relations",
        description="Defining Spatial Relations"
    )
    scenario: str = Field(
        ...,
        title="Scenario",
        description="Scenario Specifications"
    )
    background_activities: str = Field(
        ...,
        title="Background Activities",
        description="Defining Background Cars and Pedestrians"
    )

In [31]:
os.environ["OPEN_AI_API_KEY"] = "sk-gDa1bvL8Ian5Rkdq186bFbDeBf904447B4C707409e660dB6"
model_name = "gpt-3.5-turbo"
model = ChatOpenAI(
    model_name=model_name,
    max_tokens=2048,
    temperature=0,
    openai_api_key=os.environ["OPEN_AI_API_KEY"],
    openai_api_base="https://apikeyplus.com/v1"
)

In [18]:
with open("./carlaChallenge1.json", "r") as f:
    scenic_input = json.load(f) # Load the input from the JSON file

""" Scenario Description
Traffic Scenario 01.
Control loss without previous action.
The ego-vehicle loses control due to bad conditions on the road and it must recover, coming back to
its original lane.
"""
True


In [19]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301", include_final_response_prefix=True):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        #print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if "gpt-3.5" in model:
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted                                                                                                                                                                                             
    elif "gpt-4" in model:
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md \
        for information on how messages are converted to tokens.""")
    
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    if include_final_response_prefix:
        num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [28]:
def example_to_chat_messages(example):
    messages = [
        {"role": "system", "name": "example_user", 
         "content": '""" Scenario description\n%s\n"""' % example['docstring']},
        {"role": "system", "name": "example_assistant", 
         "content": '```##Map and Model##\n%s\n##Constants##\n%s\n##Moniters##\n%s\n##Defining Agent Behaviors##\n%s\n##Spatial Relations##\n%s\n##Scenario Specifications##\n%s\n##Background Activities##\n%s\n```' \
             % (example['map_and_model'], example['constants'], example['monitors'], example['behaviors'], example['spatial_relations'], example['scenario'], example['background'])}
    ]
    return messages

def get_example_num_tokens(example):
    return num_tokens_from_messages(example_to_chat_messages(example), model_name, 
                                    include_final_response_prefix=False)

def input_to_chat_messages(example):
    docstring = example if (isinstance(example, str)) else example['docstring']
    messages = [
        {"role": "user", 
         "content": '""" Scenario description\n%s\n"""' % docstring}
    ]
    return messages

def get_input_num_tokens(example, include_final_response_prefix=True):
    return num_tokens_from_messages(input_to_chat_messages(example), model_name, 
                                    include_final_response_prefix=include_final_response_prefix)


In [29]:
system_message_str="""You are a helpful agent that generates specifications for car driving scenarios in the Scenic language
Scenic is a domain-specific probabilistic programming language for modeling the environments of cyber-physical systems like robots and autonomous cars. A Scenic program defines a distribution over scenes, \
    configurations of physical objects and agents; sampling from this distribution yields concrete scenes which can be simulated to produce training or testing data. Scenic can also define (probabilistic) \
    policies for dynamic agents, allowing modeling scenarios where agents take actions over time in response to the state of the world.

Your task is to generate Scenic scenarios, each according to its corresponding description in English included as a docstring. Write each scenario in a separate code box. Follow the examples below:"""

system_role = "system" if model_name.startswith("gpt-4") else "user"
system_message = {"role": system_role, "content": system_message_str}
system_message_len = num_tokens_from_messages([system_message], model_name, include_final_response_prefix=False)

print(example_to_chat_messages(scenic_input))
print("System message tokens: %s" % system_message_len)

[{'role': 'system', 'name': 'example_user', 'content': '""" Scenario description\n""" Scenario Description\nTraffic Scenario 01.\nControl loss without previous action.\nThe ego-vehicle loses control due to bad conditions on the road and it must recover, coming back to\nits original lane.\n"""\n"""'}, {'role': 'system', 'name': 'example_assistant', 'content': "```##Map and Model##\nparamparam carla_map = 'Town01'\nmodel scenic.simulators.carla.model\n\n\n##Constants##\nEGO_MODELEGO_SPEED = 10\n\n\n##Moniters##\nNone\n##Defining Agent Behaviors##\nbehavior    do FollowLaneBehavior(speed)\n\n\n##Spatial Relations##\n\nlane = Uniform(*network.lanes)\n\nstart = new OrientedPoint on lane.centerline\nego = new Car at start,\n    with blueprint EGO_MODEL,\n    with behavior EgoBehavior(EGO_SPEED)\n\ndebris1 = new Debris following roadDirection for Range(10, 20)\ndebris2 = new Debris following roadDirection from debris1 for Range(5, 10)\ndebris3 = new Debris following roadDirection from debris2